# 🦌 SVG Defs 🏹

This notebook demonstrates how to define custom symbols and attach them to the ipyelk
diagram.

# Defining some custom arrows

The `EndpointSymbol` class allows a list of svg children to be specified. These children
define the custom svg shape. The `EndpointSymbol` also takes two other parameters,
`correction` and `offset`. The `correction` parameter adjusts the anchor endpoint and
the `offset` moves the endpoint of the calculated path.

In [ ]:
import importnb
import networkx as nx
from IPython.display import SVG

import ipyelk
from ipyelk.contrib.molds import connectors as conn
from ipyelk.contrib.molds import structures
from ipyelk.diagram import Exporter
from ipyelk.elements import EndpointSymbol, Node, NodeProperties, Symbol, SymbolSpec
from ipyelk.elements import layout_options as opt
from ipyelk.elements import shapes

In [ ]:
NodeProperties(shape=shapes.Path(use="M 5,-5 L 0,0 L 5,5")).dict()

In [ ]:
arrow = EndpointSymbol(
    identifier="arrow",
    element=Node(
        properties=NodeProperties(shape=shapes.Path(use="M 5,-5 L 0,0 L 5,5")),
    ),
    symbol_offset=shapes.Point(
        -1, 0
    ),  # moves the arrow endpoint nearer the outside of the node
    path_offset={"x": -5, "y": 0},  # shortens the path
)


circle = EndpointSymbol(
    **{
        "identifier": "circle",
        "element": {
            "children": [
                {
                    "properties": {
                        "shape": {
                            "type": "node:path",
                            "use": "M 5,-5 L 5,5",
                        }
                    }
                },
                {
                    "properties": {
                        "shape": {
                            "type": "node:round",
                            "x": 4,
                            "y": 0,
                        }
                    },
                    "width": 8,
                    "height": 8,
                },
            ]
        },
    }
)

# Wiring up ElkDiagram with def library

In [ ]:
def use_simple_custom_arrows():
    # build graph
    g = nx.Graph()

    g.add_edge(
        "n1",
        "n2",
        properties={
            "shape": {
                "start": "arrow",  # key of the shape in the def dictionary
                "end": "circle",  # key of the shape in the def dictionary
            }
        },
    )

    # configure app
    app = ipyelk.from_nx(
        graph=g,
        layout={
            "height": "100%",
        },
    )

    app.view.symbols = SymbolSpec().add(
        arrow,
        circle,
    )
    return app


if __name__ == "__main__":
    app = use_simple_custom_arrows()
    display(app)

# Shape functions

It can be helpful to parametarize the svg def shapes

In [ ]:
def Interface(identifier, r=6):
    m = 0.25 * r
    n = 0.75 * r
    return Symbol(
        identifier=identifier,
        element=Node(
            children=[
                Node(
                    properties=NodeProperties(
                        shape=shapes.Rect(width=r, height=r),
                    )
                ),
                Node(
                    properties=NodeProperties(
                        shape=shapes.Path(use=f"M 1,{m} L {n},{r/2} L {1},{n}"),
                    )
                ),
            ]
        ),
        width=r,
        height=r,
    )


final_state = Symbol(
    identifier="final_state",
    element=structures.DoubleCircle(radius=6),
    width=12,
    height=12,
)


shape_library = SymbolSpec().add(
    conn.Rhomb("composition", r=4),
    conn.Rhomb("aggregation", r=4),
    conn.Containment("containment", r=4),
    conn.StraightArrow("arrow", r=4),
    final_state,
    Interface("interface_port", r=5),
)

# Shape defs can also be used on nodes and ports

In [ ]:
def example_with_ports():
    # build graph
    g = nx.Graph()
    tree = nx.DiGraph()

    g.add_node("n1")
    g.add_node(
        "n4",
        labels=[""],  # no label will be used
        properties={
            "shape": {
                "type": "node:use",
                "use": "final_state",  # key of the shape in the def dictionary
            },
        },
    )

    g.add_node(
        "n2",
        ports=[
            {
                "id": "n2.x",
                "properties": {
                    "shape": {
                        "use": "interface_port"  # key of the shape in the def dictionary
                    }
                },
            },
            {
                "id": "n2.y",
                "properties": {
                    "shape": {
                        "use": "interface_port"  # key of the shape in the def dictionary
                    }
                },
                "width": 5,
                "height": 5,
            },
        ],
    )

    g.add_edge(
        "n1",
        "n2",
        sourcePort="x",
        targetPort="x",
        properties={
            "shape": {
                "start": "containment",
                "end": "arrow",
            }
        },
    )

    g.add_edge(
        "n1",
        "n3",
        properties={
            "shape": {
                "start": "aggregation",
                "end": "arrow",
            }
        },
    )

    g.add_edge(
        "n2",
        "n3",
        properties={
            "shape": {
                "start": "composition",
            }
        },
    )
    g.add_node(
        "decision", width=30, height=15, properties={"shape": {"type": "node:diamond"}}
    )
    g.add_node("activity", properties={"shape": {"type": "node:round"}})
    g.add_edge(
        "n4",
        "n1",
    )

    g.add_node("other shapes")
    tree.add_edge("other shapes", "decision")
    tree.add_edge("other shapes", "activity")
    tree.add_edge("other shapes", "n4")

    # configure app
    app = ipyelk.from_nx(
        graph=g,
        hierarchy=tree,
        layout={"height": "100%"},
        style={
            " .elkarrow.aggregation": {"fill": "var(--jp-elk-node-stroke)"},
            " .final_state > g > g > g:nth-child(2)": {
                "fill": "var(--jp-elk-node-stroke)"
            },
        },
    )

    app.view.symbols = shape_library

    return app


if __name__ == "__main__":
    app = example_with_ports()
    display(app)

# Exported SVG image still maintains refs

In [ ]:
def exporter_example(app):
    import ipywidgets as W

    exporter = ipyelk.diagram.export.Exporter(diagram=app, strip_ids=False)
    out = W.Output()

    def update(change=None):
        out.clear_output()
        with out:
            display(SVG(exporter.value))

    exporter.observe(update, "value")
    update()
    return out


if __name__ == "__main__":
    app = example_with_ports()
    out = exporter_example(app)
    display(app)
    display(out)